In [1]:
# -*- encoding: utf-8 -*-
'''
@File    :   address_API.py
@Time    :   2020/09/07 15:57:42
@Author  :   DataMagician 
@Version :   1.0
@Contact :   408903228@qq.com
'''

# here put the import lib
import requests
import json
import pandas as pd

def get_api_function(*address):
    '''
    调用地图API获取待查询地址专属url
    最高查询次数30w/天，最大并发量160/秒
    '''
    ak = 'Vhzgfjih2w3YY0WUPuPdKiMQKwIor9rp'
    if len(address) < 1:
        return None
    else:
        for add in address:   
            url = 'http://api.map.baidu.com/geocoding/v3/?address={inputAddress}&output=json&ak={myAk}'.format(inputAddress=add,myAk=ak)  
            yield url
            
def get_position_function(url):
    '''返回经纬度信息'''
    res = requests.get(url)
    json_data = json.loads(res.text)

    if json_data['status'] == 0:
        lat = json_data['result']['location']['lat'] #经度
        lng = json_data['result']['location']['lng'] #纬度
        precise = json_data['result']['precise'] #精确性
        confidence = json_data['result']['confidence'] #置信度
        comprehension = json_data['result']['comprehension'] #解释性
        level = json_data['result']['level'] #分类等级
    else:
        print("Error output!")
        return json_data['status']

    
    return lat,lng,precise,confidence,comprehension,level


def get_strings_function(address):
    ''''''
    for add in address:
        add_url = list(get_api_function(add))[0]
        print(add_url)
        try:
            lat,lng,precise,confidence,comprehension,level = get_position_function(add_url) 
            print("地址：{0}|经度:{1}|纬度:{2}|精度:{3}|置信度:{4}|解释性:{5}|分类等级:{6}".format(add,lat,lng,precise,confidence,comprehension,level))
        except Error as e:
            print(e)

def getlonglat_function(address):
    ''''''
    column = {"地址":[],"经度":[],"纬度":[],"精度":[],"置信度":[],"解释性":[],"分类等级":[]}
    n = -1
    for add in address:
        column["地址"].append(add)
        lat,lng,precise,confidence,comprehension,level  = get_position_function(list(get_api_function(add))[0])
        column["纬度"].append(lat)
        column["经度"].append(lng)
        column["精度"].append(precise)
        column["置信度"].append(confidence)
        column["解释性"].append(comprehension)
        column["分类等级"].append(level)           
    return column






In [2]:
str_ = "灵隐寺-浙江省博物馆-浙江省图书馆-晓书馆-蚂蚁金服-浙江省金华市东阳市横店镇-西湖景区".split("-")
str_

['灵隐寺', '浙江省博物馆', '浙江省图书馆', '晓书馆', '蚂蚁金服', '浙江省金华市东阳市横店镇', '西湖景区']

In [3]:
df = pd.DataFrame(getlonglat_function(str_))
df

,地址,经度,纬度,精度,置信度,解释性,分类等级
0,灵隐寺,106.464037,30.097301,0,50,0,NoClass
1,浙江省博物馆,120.149943,30.257342,0,50,0,旅游景点
2,浙江省图书馆,120.160075,30.293436,0,50,0,NoClass
3,晓书馆,116.257071,39.916085,0,70,0,教育
4,蚂蚁金服,116.466544,39.924502,0,50,0,NoClass
5,浙江省金华市东阳市横店镇,120.294085,29.161733,0,25,100,乡镇
6,西湖景区,120.164255,30.264728,0,50,100,旅游景点


In [4]:
import json
import plotly.express as px
import plotly.graph_objects as go
token = 'pk.eyJ1IjoiYmxhY2tzaGVlcHdhbGwwMzA1IiwiYSI6ImNrMHo5ZnQxYjBjbG8zbm84b3hrb25vb24ifQ.K8tcDjJDsPcjdYFTSVgTxw'
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(mode='markers', 
                                 lon = df.经度,
                                 lat = df.纬度,
                                 hovertext = df.地址,
                                 hoverinfo = 'text',
                                 marker = {'symbol': 'marker', 'size': 10}
                                 ))
	
fig.update_layout(mapbox = {'accesstoken': token, 'center': {'lon': 30.212767, 'lat': 120.025229	}, 'zoom': 10.5},
                  margin = {'l': 0, 'r': 0, 't': 0, 'b': 0})
fig.show()

In [5]:
KMpoint = df[df["地址"]=="浙江省图书馆"]
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
#token='pk.eyJ1Ijoic3l2aW5jZSIsImEiOiJjazZrNTcwY3kwMHBrM2txaGJqZWEzNWExIn0.tLQHY_OoiR2NMxnYHXUBAA'
token = 'pk.eyJ1IjoiYmxhY2tzaGVlcHdhbGwwMzA1IiwiYSI6ImNrMHo5ZnQxYjBjbG8zbm84b3hrb25vb24ifQ.K8tcDjJDsPcjdYFTSVgTxw'
token = 'pk.eyJ1IjoibWFubWFuemhhbmciLCJhIjoiY2thMDRkbGM3MDh0aDNybjZpM3hyam5yOCJ9.TOaUXDGA4hzIoAowDRSOqw'
university = df
university.置信度 = university.置信度*1000
university.经度.mean(),university.纬度.mean()
px.set_mapbox_access_token(token)
fig4 = px.scatter_mapbox(university
                         , lat="经度"
                         , lon="纬度"
                         , color="置信度"
                         , size="解释性"
                         , color_continuous_scale=px.colors.cyclical.IceFire
                         , size_max=6
                         , zoom=9
                         ,center = {'lon': 39.924502, 'lat': 120.025229	})
fig4.show()
